In [125]:
import pandas as pd
import numpy as np
import re 
df_ap_clean = pd.read_json('viva_ap.json')


In [126]:
# Função para extrair as partes do endereço
def extrair_endereco(endereco):
    partes = endereco.split(' - ')
    if len(partes) == 3:
        address = partes[0].strip()
        region_city_parts = partes[1].split(',')
        regionName = region_city_parts[0].strip() if len(region_city_parts) > 0 else ''
        city = region_city_parts[1].strip() if len(region_city_parts) > 1 else ''
    elif len(partes) == 2:
        address = ''
        region_city_parts = partes[0].split(',')
        regionName = region_city_parts[0].strip() if len(region_city_parts) > 0 else ''
        city = region_city_parts[1].strip() if len(region_city_parts) > 1 else ''
    else:
        address = ''
        regionName = partes[0].strip()
        city = 'Belo Horizonte'
    
    return pd.Series([address, regionName, city])

In [127]:
def processar_dados_apartamentos(df):
    # Aplicando a função ao DataFrame
    df[['endereco', 'bairro', 'cidade']] = df['endereco'].apply(extrair_endereco)
    
    # Removendo duplicatas
    df = df.drop_duplicates(subset=['valor', 'endereco','quartos','garagem','banheiro','area(m2)','condominio'])
    
    # Filtrando apenas aps de BH
    df = df[df['cidade'] == 'Belo Horizonte']
    del df['cidade']
    
    # Tratando o campo garagem
    df.loc[:, 'garagem'] = df['garagem'].apply(lambda x: '0' if x == '--' else x)
    df.loc[:, 'garagem'] = df['garagem'].astype(int)
    
    return df

In [128]:
df_ap_clean = processar_dados_apartamentos(df_ap_clean)

In [129]:
def remove_outliers(df, column, z_threshold=10):
    """
    Remove outliers de uma coluna específica de um DataFrame com base em um valor de Z-score.

    Parâmetros:
    df (pd.DataFrame): O DataFrame de entrada.
    column (str): O nome da coluna para detectar e remover outliers.
    z_threshold (float): O limiar do Z-score para definir outliers (padrão é 10).

    Retorna:
    pd.DataFrame: O DataFrame sem os outliers na coluna especificada.
    """
    mean_val = df[column].mean()
    std_dev_val = df[column].std()

    # Calcular o Z-score
    df = df.copy()
    df['z_score'] = (df[column] - mean_val) / std_dev_val
    
    
    # Detectar e remover outliers
    df = df[df['z_score'] <= z_threshold]
    del df['z_score']

    return df

In [130]:
# Remover outliers das colunas 'valor', 'condominio' e 'iptu'
df_ap_clean = remove_outliers(df_ap_clean, 'valor', 5)


In [131]:
df_ap_clean

,valor,endereco,quartos,garagem,banheiro,area(m2),condominio,bairro
0,4000,"Rua Odilon Braga, 545",4,3,4,159,1000,Anchieta
1,3500,"Rua Lauro Ferreira, 101",4,2,3,144,1000,Buritis
2,3200,"Rua Tomé de Souza, 1234",3,2,3,135,800,Savassi
3,2800,"Rua São Paulo, 1801",2,2,2,120,700,Lourdes
4,2800,"Rua Professor Arduino Bolivar, 91",4,2,4,140,1200,Santo Antônio
...,...,...,...,...,...,...,...,...
3988,20000,"Rua Rodrigo Otávio Coutinho, 320",4,4,1,385,2400,Belvedere
3989,15000,"Rua Maestro Arthur Bosmans, 55",4,4,5,204,2975,Belvedere
3990,6000,,4,2,4,304,1800,Floresta
3991,15000,"Rua Levy Lafetá, 109",3,6,4,260,1950,Belvedere


In [132]:
df_cobertura = pd.read_json('viva_cob.json')

df_cobertura = processar_dados_apartamentos(df_cobertura)

In [133]:
#Analise de outliers foi feita e não foi encontrado nenhum outlier 

In [134]:
df_cobertura

,valor,endereco,quartos,garagem,banheiro,area(m2),condominio,bairro
0,3100,Rua Cecília Fonseca Coutinho,3,2,3,264,572,Castelo
1,3000,"Rua Santo Antônio do Monte, 670",2,1,3,120,700,Santo Antônio
2,6000,Rua General Andrade Neves,4,4,4,230,764,Gutierrez
3,2900,"Rua Guaxupé, 230",2,2,3,110,550,Serra
4,4500,"Avenida dos Bandeirantes, 459",4,3,3,220,1200,Sion
...,...,...,...,...,...,...,...,...
633,10000,Rua Maranhão,3,4,5,300,3174,Funcionários
634,20000,,4,4,1,357,3517,Belvedere
635,20000,,4,4,4,344,3500,Belvedere
636,20000,Rua Castelo da Beira,4,3,3,230,680,Castelo


In [135]:
df_cobertura['penthouse'] =1
df_ap_clean['penthouse'] = 0

In [136]:
df = pd.concat([df_ap_clean, df_cobertura], ignore_index=True)
df

,valor,endereco,quartos,garagem,banheiro,area(m2),condominio,bairro,penthouse
0,4000,"Rua Odilon Braga, 545",4,3,4,159,1000,Anchieta,0
1,3500,"Rua Lauro Ferreira, 101",4,2,3,144,1000,Buritis,0
2,3200,"Rua Tomé de Souza, 1234",3,2,3,135,800,Savassi,0
3,2800,"Rua São Paulo, 1801",2,2,2,120,700,Lourdes,0
4,2800,"Rua Professor Arduino Bolivar, 91",4,2,4,140,1200,Santo Antônio,0
...,...,...,...,...,...,...,...,...,...
4233,10000,Rua Maranhão,3,4,5,300,3174,Funcionários,1
4234,20000,,4,4,1,357,3517,Belvedere,1
4235,20000,,4,4,4,344,3500,Belvedere,1
4236,20000,Rua Castelo da Beira,4,3,3,230,680,Castelo,1


In [137]:
def remove_duplicates_with_priority(df):
    """
    Remove duplicatas de um DataFrame com base em colunas específicas,
    mantendo as linhas onde a coluna 'penthouse' é 1.
    
    """
    # Ordenar para que as linhas com 'penthouse' == 1 venham primeiro
    df = df.sort_values(by='penthouse', ascending=False)

    # Remover duplicatas mantendo a primeira ocorrência (onde 'penthouse' == 1)
    df = df.drop_duplicates(subset=['valor', 'bairro', 'endereco','quartos','garagem','banheiro','area(m2)','condominio'])

    return df

In [138]:
#Removendo os valores que tem TODOS os dados iguais 
df = remove_duplicates_with_priority(df)


In [139]:
#Conferindo se tem algum dado repetido 
df.duplicated(subset=['valor', 'bairro', 'endereco','quartos','garagem','banheiro','area(m2)','condominio'], keep=False).value_counts()

False    4225
Name: count, dtype: int64

In [140]:
# Renomeando colunas 
df = df.rename(columns={
    'valor': 'price',
    'bairro': 'regionName',
    'endereco': 'address',
    'quartos': 'rooms',
    'garagem': 'parking',
    'banheiro': 'toilets',
    'area(m2)': 'size',
    'condominio': 'iptuPlusCondominium'
})


In [141]:
df

,price,address,rooms,parking,toilets,size,iptuPlusCondominium,regionName,penthouse
4222,16000,,3,4,3,200,3200,Lourdes,1
4221,20000,"Avenida Paulo Camilo Pena, 609",4,4,5,357,3500,Belvedere,1
4206,10000,,4,3,3,200,1700,Savassi,1
4237,30000,"Rua Cônego Rocha Franco, 28",4,4,5,401,3297,Gutierrez,1
3708,4000,"Rua Sílvio Menicucci, 170",3,3,3,150,421,Buritis,1
...,...,...,...,...,...,...,...,...,...
603,1160,,2,1,1,80,0,Sao Damiao,0
604,2600,Avenida Flávio dos Santos,3,2,2,90,325,Floresta,0
605,3100,Rua Romualdo Lopes Cançado,3,2,3,120,550,Castelo,0
606,2600,,3,2,3,78,360,Nova Floresta,0


In [143]:
df.to_json('dados/viva.json', orient='records', force_ascii=False, indent=4)